# The CSV file challenge


In [4]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None

In [5]:
from azure.storage.blob import BlockBlobService
block_blob_service = BlockBlobService(
 account_name='asadatalake01', account_key='c9QiMiFm2v44KQD0+g+gnDArV7/Kw5UO/+3EIsELimWy+n2hAxOMxS2ZuMN88nYIsx24cJqN4kWrKk/PgteMww==')
file_content = block_blob_service.get_blob_to_text('wwi-02', 'sale-poc/sale-20170502.csv')

In [6]:

tokens = file_content.content.split(',')
 
print(f'Found {len(tokens)} tokens in content.')


Found 10573487 tokens in content.

In [7]:
[tokens[i] for i in np.arange(0, 10)] 
[tokens[i] for i in np.arange(0, 15)] 
[tokens[i] for i in np.arange(0, 20)] 

['586682a8-db73-4e45-a376-c7553a44b248', '2', '610', '4', '35.790000000000000000', '157.4760000000000127151622564', '20170502', '53.63600000000000433075797446', '23', '17', '9559', '586682a8-db73-4e45-a376-c7553a44b248', '2', '1321', '3', '22.080000000000000000', '72.86400000000000588329385209', '20170502', '19.66800000000000158806301442', '23']

In [8]:
row_list = []
 
max_index = 11
while max_index <= len(tokens):
 
 row = [tokens[i] for i in np.arange(max_index - 11, max_index)] 
 row_list.append(row)
 
 max_index += 11

In [10]:
row_list[0:10]

[['586682a8-db73-4e45-a376-c7553a44b248', '2', '610', '4', '35.790000000000000000', '157.4760000000000127151622564', '20170502', '53.63600000000000433075797446', '23', '17', '9559'], ['586682a8-db73-4e45-a376-c7553a44b248', '2', '1321', '3', '22.080000000000000000', '72.86400000000000588329385209', '20170502', '19.66800000000000158806301442', '23', '17', '9559'], ['586682a8-db73-4e45-a376-c7553a44b248', '2', '4364', '1', '16.150000000000000000', '17.76500000000000143440814782', '20170502', '5.896000000000000476063632959', '23', '17', '9559'], ['586682a8-db73-4e45-a376-c7553a44b248', '2', '4691', '2', '35.840000000000000000', '78.84800000000000636646291241', '20170502', '24.20000000000000195399252334', '23', '17', '9559'], ['586682a8-db73-4e45-a376-c7553a44b248', '2', '4691', '3', '35.840000000000000000', '118.2720000000000095496943686', '20170502', '36.30000000000000293098878501', '23', '17', '9559'], ['586682a8-db73-4e45-a376-c7553a44b248', '2', '2392', '1', '32.900000000000000000', '

In [11]:
df = pd.DataFrame(row_list, columns=['TransactionId','CustomerId', 'ProductId', 'Quantity', 'Price', 'TotalAmount', 'TransactionDateId', 'ProfitAmount', 'Hour', 'Minute', 'StoreId'])

In [12]:
df.head(100)

TransactionId CustomerId ProductId Quantity  \
0   586682a8-db73-4e45-a376-c7553a44b248          2       610        4   
1   586682a8-db73-4e45-a376-c7553a44b248          2      1321        3   
2   586682a8-db73-4e45-a376-c7553a44b248          2      4364        1   
3   586682a8-db73-4e45-a376-c7553a44b248          2      4691        2   
4   586682a8-db73-4e45-a376-c7553a44b248          2      4691        3   
5   586682a8-db73-4e45-a376-c7553a44b248          2      2392        1   
6   586682a8-db73-4e45-a376-c7553a44b248          2      1347        2   
7   586682a8-db73-4e45-a376-c7553a44b248          2       402        2   
8   586682a8-db73-4e45-a376-c7553a44b248          2        56        3   
9   586682a8-db73-4e45-a376-c7553a44b248          2       157        2   
10  586682a8-db73-4e45-a376-c7553a44b248          2        78        1   
11  21cacd35-336f-442e-8afd-8cb637678c56          4       238        1   
12  21cacd35-336f-442e-8afd-8cb637678c56          4       138    

# The country challenge


In [4]:
%%pyspark
data_path = spark.read.load('abfss://wwi-02@asadatalake01.dfs.core.windows.net/data-generators/generator-customer.csv'
, format='csv'
, header=True
)
display(data_path.limit(10))

There seems to be an issue with the address containing the separator character, but it also looks like the address is correctly quoted. Let's reload the data with a quote character specification:


In [5]:
%%pyspark
data_path = spark.read.load('abfss://wwi-02@asadatalake01.dfs.core.windows.net/data-generators/generator-customer.csv'
, format='csv'
, header=True
, quote='\''
)
display(data_path.limit(10))

In [20]:
cust_df = data_path.toPandas()

In [21]:
cust_df[['Addr_Split_1', 'Addr_Split_2']] = cust_df.Address.str.split(",",expand=True) 
temp1 = cust_df.Addr_Split_1.str.split(" ", expand=True)
cust_df['Address_PostalCode'] = temp1[0]
cust_df['Address_Street'] = temp1[1] + ' ' + temp1[2]

In [28]:
cust_df.Addr_Split_2

0                                          Mazieton Guyana
1                                       Ziemeborough Nauru
2                                      West Nadia Slovenia
3                                        Libbiefort Taiwan
4                                  West Paolofort Dominica
5                                        Lake Ebony Taiwan
6                                East Cristianview Reunion
7                                            Janport Malta
8                        Jakaylashire Syrian Arab Republic
9                                 Pansyhaven Liechtenstein
10                                     Lake Hector Morocco
11          Isabelleburgh Lao People's Democratic Republic
12                                     New Nonaberg Angola
13                              Port Lesterborough Armenia
14             New Kacie Heard Island and McDonald Islands
15         Olsonmouth United States Minor Outlying Islands
16                                  Alessandraview Roman

Let's find a list of country names from a public resource (e.g. https://gist.github.com/marijn/396531/188caa065e3cd319fed7913ee3eecf5eec541918). Save it in the `tempdata` folder of the data lake as `countries.txt`. Note that we kept only the country names (removed the country codes).


In [62]:
countries = spark.read.load('abfss://tempdata@asadatalake01.dfs.core.windows.net/Country.txt', format='text')
country_df = countries.toPandas()
country_list = list(set(country_df['value'].str.lower()))

In [50]:
temp2 = cust_df.Addr_Split_2.str.split(" ", expand=True)
temp2 = temp2.fillna(value='')
temp2['Address_City'] = None
temp2['Address_Country'] = None

In [65]:
temp2['Candidate_Country_1'] = temp2[1] + ' ' + temp2[2] + ' ' + temp2[3] + ' ' + temp2[4] + ' ' + temp2[5] + ' ' + temp2[6] + ' ' + temp2[7] + ' ' + temp2[8] + ' ' + temp2[9] + ' ' + temp2[10]
temp2['Candidate_Country_1'] = temp2['Candidate_Country_1'].str.strip().str.lower()
temp2['Candidate_Country_2'] = temp2[2] + ' ' + temp2[3] + ' ' + temp2[4] + ' ' + temp2[5] + ' ' + temp2[6] + ' ' + temp2[7] + ' ' + temp2[8] + ' ' + temp2[9] + ' ' + temp2[10]
temp2['Candidate_Country_2'] = temp2['Candidate_Country_2'].str.strip().str.lower()
temp2['Candidate_Country_3'] = temp2[3] + ' ' + temp2[4] + ' ' + temp2[5] + ' ' + temp2[6] + ' ' + temp2[7] + ' ' + temp2[8] + ' ' + temp2[9] + ' ' + temp2[10]
temp2['Candidate_Country_3'] = temp2['Candidate_Country_3'].str.strip().str.lower()
temp2['Candidate_Country_4'] = temp2[4] + ' ' + temp2[5] + ' ' + temp2[6] + ' ' + temp2[7] + ' ' + temp2[8] + ' ' + temp2[9] + ' ' + temp2[10]
temp2['Candidate_Country_4'] = temp2['Candidate_Country_4'].str.strip().str.lower()

In [66]:
print(temp2['Candidate_Country_1'].isin(country_list).sum())
print(temp2['Candidate_Country_2'].isin(country_list).sum())
print(temp2['Candidate_Country_3'].isin(country_list).sum())
print(temp2['Candidate_Country_4'].isin(country_list).sum())

0
368938
518161
6888

In [67]:
temp2.loc[temp2['Candidate_Country_2'].isin(country_list), 'Address_Country'] = temp2['Candidate_Country_2']
temp2.loc[temp2['Candidate_Country_3'].isin(country_list), 'Address_Country'] = temp2['Candidate_Country_3']
temp2.loc[temp2['Candidate_Country_4'].isin(country_list), 'Address_Country'] = temp2['Candidate_Country_4']

In [71]:
temp2[temp2['Address_Country'].isnull()]

0                 1              2            3           4  \
3               Libbiefort         Taiwan                            
5                     Lake          Ebony       Taiwan               
18         Brooklynborough       Tanzania                            
32                    Lake     Moisesfort   Democratic    People's   
36             Jacobiburgh          Saint   Barthelemy               
41           Reynoldshaven         Virgin      Islands     British   
45               Taramouth     Kazakhstan                            
63                   South  Kelleyborough        Saint  Barthelemy   
64            Estevanville         Taiwan                            
78                   South      Myrnafurt       Virgin     Islands   
103                   West      Myronstad     Pitcairn     Islands   
107              Tomasberg      Macedonia                            
109              Calihaven        British       Indian       Ocean   
111                 Ianton 

In [63]:
country_list.sort()
country_list

['afghanistan', 'albania', 'algeria', 'american samoa', 'andorra', 'angola', 'anguilla', 'antarctica', 'antigua and barbuda', 'argentina', 'armenia', 'aruba', 'australia', 'austria', 'azerbaijan', 'bahamas', 'bahrain', 'bangladesh', 'barbados', 'belarus', 'belgium', 'belize', 'benin', 'bermuda', 'bhutan', 'bolivia', 'bosnia and herzegovina', 'botswana', 'bouvet island', 'brazil', 'british indian ocean territory', 'brunei darussalam', 'bulgaria', 'burkina faso', 'burundi', 'cambodia', 'cameroon', 'canada', 'cape verde', 'cayman islands', 'central african republic', 'chad', 'chile', 'china', 'christmas island', 'cocos (keeling) islands', 'colombia', 'comoros', 'congo', 'congo, the democratic republic of the', 'cook islands', 'costa rica', "cote d'ivoire", 'croatia', 'cuba', 'cyprus', 'czech republic', 'denmark', 'djibouti', 'dominica', 'dominican republic', 'east timor', 'ecuador', 'egypt', 'el salvador', 'equatorial guinea', 'eritrea', 'estonia', 'ethiopia', 'falkland islands (malvinas)

In [70]:
country_list.append('taiwan')

In [31]:
temp2[(~temp2[9].isnull()) & (temp2['Address_City'].isnull())]

0                1               2      3          4      5    6  \
117                  East     Fabianville  South    Georgia    and  the   
294                  Lake      Bartonberg  South    Georgia    and  the   
458                 North            Myah  South    Georgia    and  the   
596           Wisokyshire      Antarctica   (the  territory  South   of   
748                  West      Wernerside  South    Georgia    and  the   
1245                 West      Electafort  South    Georgia    and  the   
1282                South          Lenora  South    Georgia    and  the   
1292          Assuntaland      Antarctica   (the  territory  South   of   
1418            Bayertown      Antarctica   (the  territory  South   of   
1470          Jaylinmouth      Antarctica   (the  territory  South   of   
1535                 Port         Damaris  South    Georgia    and  the   
1588           Vernieland      Antarctica   (the  territory  South   of   
1819                North       

In [34]:
temp2[~temp2['Address_City'].isnull()]

0       1               2           3     4          5      6   7   8  \
161        North    Rhiannonside  Antarctica  (the  territory  South  of  60   
259         West          Colten  Antarctica  (the  territory  South  of  60   
266        South           Hilma  Antarctica  (the  territory  South  of  60   
293         West         Toystad  Antarctica  (the  territory  South  of  60   
566         East         Madalyn  Antarctica  (the  territory  South  of  60   
1320        Port        Hankfurt  Antarctica  (the  territory  South  of  60   
1321        West     Marquestown  Antarctica  (the  territory  South  of  60   
1480       North         Darrion  Antarctica  (the  territory  South  of  60   
2063        Port        Hankfurt  Antarctica  (the  territory  South  of  60   
2139        East     Thereseside  Antarctica  (the  territory  South  of  60   
2263        Lake  Alexandroburgh  Antarctica  (the  territory  South  of  60   
2682       South       Pearlfurt  Antarctica  (